Limitations de la plage de dates :

startDateet endDatedoit suivre le format ISO 8601 :

Format de base : "yyyy-MM-dd" (par défaut minuit).

Format complet avec décalage horaire : "yyyy-MM-dd'T'HH:mm:ssXXX" (doit être codé en URL, par exemple, 2023-01-01T00:00:00%2B02:00).

Comportement:

Les deux startDatesont endDateinclus dans la réponse.

Si endDateest omis, la valeur par défaut est la date actuelle ou la plage maximale autorisée après startDate.

Si startDateest omis, la valeur par défaut est la date la plus ancienne autorisée dans la plage de l'ensemble de données.

Si les deux sont omis, l'API définit automatiquement :

endDate = current time

startDate = endDate - max allowed range (day/week/month, depending on the dataset)

In [2]:
import requests
import pandas as pd
import csv
from datetime import datetime, timedelta
import time 

## Test initial de l'API

### Explication du code
Objectif : Comprendre la structure des données retournées par l'API Elhub avant d'implémenter la collecte automatisée.

### Paramètres de test utilisés :
Dataset : CONSUMPTION_PER_GROUP_MUNICIPALITY_HOUR - Consommation par groupe et municipalité par heure

Date de début : 2023-05-03T20:00:00+02:00 - 3 mai 2023 à 20h00

Date de fin : 2023-05-04T00:00:00+02:00 - 4 mai 2023 à 00h00 (période de 4 heures)

Groupe de consommation : private - Consommation privée

Période choisie : Une courte période (4 heures) pour limiter le volume de données lors de cette exploration initiale.

### Résultat attendu
Ce test permet de :

1.Vérifier la connectivité à l'API

2.Comprendre le format JSON retourné

3.Identifier les champs disponibles dans les données

4.Valider les paramètres de requête

### Prochaines étapes
Une fois la structure des données comprise grâce à ce test, l'objectif est d'automatiser la collecte sur des périodes plus longues et de traiter les données de manière systématique.

Note : Ce code est un exemple d'exploration tiré directement du site de l'API Elhub pour des fins de test et de compréhension de la structure des données.

### Code d'exploration

In [ ]:
# Define the headers for the request
headers = {}
# Make a request to the Elhub API to get municipalities data
response = requests.get('https://api.elhub.no/energy-data/v0/municipalities?dataset=CONSUMPTION_PER_GROUP_MUNICIPALITY_HOUR&startDate=2023-05-03T20:00:00%2B02:00&endDate=2023-05-04T00:00:00%2B02:00&consumptionGroup=private', headers=headers)
if response.status_code == 200: # Check if the request was successful
    print(response.json())
else:
    print('Error:', response.status_code)

{'meta': {'created': '2025-05-26T13:49:09+02:00', 'lastUpdated': '2025-03-29T01:42:43+01:00'}, 'links': {'self': 'https://api.elhub.no/energy-data/v0/municipalities?consumptionGroup=private&dataset=CONSUMPTION_PER_GROUP_MUNICIPALITY_HOUR&endDate=2023-05-04T00%3A00%3A00%2B02%3A00&startDate=2023-05-03T20%3A00%3A00%2B02%3A00'}, 'data': [{'type': 'municipality', 'id': '*', 'attributes': {'municipalityNumber': '*', 'name': '*', 'nameNo': '*', 'consumptionPerGroupMunicipalityHour': []}}, {'type': 'municipality', 'id': '0000', 'attributes': {'municipalityNumber': '0000', 'name': 'Ukjent tilhørighet', 'nameNo': 'Ukjent tilhørighet', 'consumptionPerGroupMunicipalityHour': [{'startTime': '2023-05-03T20:00:00+02:00', 'endTime': '2023-05-03T21:00:00+02:00', 'consumptionGroup': 'private', 'quantityKwh': 2094.047, 'lastUpdatedTime': '2025-03-29T01:42:43+01:00', 'meteringPointCount': 1363}, {'startTime': '2023-05-03T21:00:00+02:00', 'endTime': '2023-05-03T22:00:00+02:00', 'consumptionGroup': 'private

In [ ]:
# Constants
dataset = "CONSUMPTION_PER_GROUP_MUNICIPALITY_HOUR" # Dataset identifier in the API
consumption_groups = ["private", "industry", "business"] # List of consumption groups to retrieve data for
base_url = "https://api.elhub.no/energy-data/v0/municipalities"  # Base URL for the API endpoint

# Start and end dates for data retrieval
start_date = datetime(2021, 1, 1) # Start date for data collection autorised by the API
end_limit = datetime.now() # Current date and time

# Loop over each consumption group to create separate CSV files
for group in consumption_groups:
    file_name = f"municipalities_group_{group}_consumption.csv" # Name of the output CSV file for the current group
    print(f"\nStarting data extraction for consumption group: {group}") # Print the current group being processed
    
    # Ensemble pour collecter automatiquement tous les en-têtes possibles
    all_fieldnames = set()
    all_rows = []  # Stocker toutes les lignes pour les écrire après avoir collecté les en-têtes
    
    current_date = start_date # Initialize current date to start date will be incremented day by day
    
    # Loop day by day until end_limit
    while current_date < end_limit:
        end_date = current_date + timedelta(days=1) # Define the end date for the current day 1 day later because the API requires a range of dates in one request
        
        # Format dates as ISO 8601 (YYYY-MM-DD)
        start_str = current_date.strftime("%Y-%m-%d") # Format start date to string because the API requires date strings 
        end_str = end_date.strftime("%Y-%m-%d")
        
        # Construct API URL with parameters for current group and date range
        url = (
            f"{base_url}?dataset={dataset}"
            f"&startDate={start_str}"
            f"&endDate={end_str}"
            f"&consumptionGroup={group}"
        ) # Construct the full URL for the API request
        print(f"API call: {start_str} → {end_str} for group '{group}'")  # Print the API call being made
        
        try:  # Make the API request in a try-except block to handle potential errors
            response = requests.get(url)   # Send GET request to the API
            if response.status_code == 200:   # Check if the request was successful 200 because it means OK in HTTP status codes
                json_full = response.json()   # Parse the JSON response from the API because the API returns data in JSON format
                data = json_full.get("data", []) # Extract the "data" field from the JSON response, defaulting to an empty list if not present
                
                # Extract relevant fields from the JSON response
                for item in data: # Loop through each item in the data list
                    if not isinstance(item, dict):   # Check if the item is a dictionary 
                        continue
                    
                    # Structure de base de l'item before adding attributes 
                    base_row = {
                        "type": item.get("type"), 
                        "id": item.get("id")
                    }
                    
                    attr = item.get("attributes", {}) # Extract the "attributes" field, defaulting to an empty dictionary if not present
                    
                    # Ajouter automatiquement tous les attributs au niveau municipality 
                    for attr_key, attr_value in attr.items():  # Loop through each attribute in the attributes dictionary
                        if attr_key != "consumptionPerGroupMunicipalityHour":  # Skip the nested consumption data
                            base_row[attr_key] = attr_value 
                    
                    # Loop through the hourly consumption records
                    for record in attr.get("consumptionPerGroupMunicipalityHour", []): 
                        # Créer une ligne complète en combinant les infos de base et les données horaires
                        row = {**base_row}  # Copier les données de base
                        
                        # Ajouter automatiquement toutes les données du record
                        for record_key, record_value in record.items():
                            row[record_key] = record_value   # Add each record key-value pair to the row
                        
                        # Ajouter toutes les clés trouvées à notre ensemble d'en-têtes
                        all_fieldnames.update(row.keys())
                        all_rows.append(row)
                
                if data:
                    print(f"  {len(data)} records processed for date {start_str}") # Print the number of records processed for the current date
                else:
                    print(f"  No data for date {start_str}") # Print if no data was found for the current date
            else:
                print(f"API call failed with status code: {response.status_code}") # Print error if the API call was not successful
                
        except Exception as e:
            print(f"API error: {e}") # Print any exceptions that occur during the API call
        
        # Move to next day
        current_date = end_date # Increment the current date by one day to process the next date
        time.sleep(1)  # Sleep 1 second to avoid rate limiting because the API may have rate limits on requests
    
    # Écrire toutes les données dans le fichier CSV avec les en-têtes détectées automatiquement
    if all_rows: # Check if there are any rows to write
        with open(file_name, mode="w", newline="", encoding="utf-8") as file: # Open the file for writing
            # Trier les en-têtes pour avoir un ordre cohérent
            sorted_fieldnames = sorted(all_fieldnames) 
            writer = csv.DictWriter(file, fieldnames=sorted_fieldnames)
            writer.writeheader() # Write the header row with sorted fieldnames
            writer.writerows(all_rows)
        
        print(f"Data saved to file: {file_name}")
        print(f"Detected fieldnames: {sorted_fieldnames}")
        print(f"Total rows: {len(all_rows)}")
    else:
        print(f"No data found for group: {group}") # Print if no data was found for the current group


Starting data extraction for consumption group: private
API call: 2025-07-13 → 2025-07-14 for group 'private'
  361 records processed for date 2025-07-13
API call: 2025-07-14 → 2025-07-15 for group 'private'
  361 records processed for date 2025-07-14
API call: 2025-07-15 → 2025-07-16 for group 'private'
  361 records processed for date 2025-07-15
API call: 2025-07-16 → 2025-07-17 for group 'private'
  361 records processed for date 2025-07-16
API call: 2025-07-17 → 2025-07-18 for group 'private'
  361 records processed for date 2025-07-17
Data saved to file: municipalities_group_private_consumption.csv
Detected fieldnames: ['consumptionGroup', 'endTime', 'id', 'lastUpdatedTime', 'meteringPointCount', 'municipalityNumber', 'name', 'nameNo', 'quantityKwh', 'startTime', 'type']
Total rows: 25776

Starting data extraction for consumption group: industry
API call: 2025-07-13 → 2025-07-14 for group 'industry'
  361 records processed for date 2025-07-13
API call: 2025-07-14 → 2025-07-15 for

In [ ]:
df_collect_private_consump=pd.read_csv('municipalities_group_private_consumption.csv')
df_collect_private_consump

In [ ]:
df_collect_private_consump = df_collect_private_consump.drop(columns=['lastUpdatedTime', 'nameNo']) # Drop unnecessary columns

In [ ]:

df_collect_private_consump

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
0,municipality,0,0,Ukjent tilhørighet,2021-01-01T00:00:00+01:00,2021-01-01T01:00:00+01:00,private,3804.885,1721
1,municipality,0,0,Ukjent tilhørighet,2021-01-01T01:00:00+01:00,2021-01-01T02:00:00+01:00,private,3781.050,1721
2,municipality,0,0,Ukjent tilhørighet,2021-01-01T02:00:00+01:00,2021-01-01T03:00:00+01:00,private,3694.389,1721
3,municipality,0,0,Ukjent tilhørighet,2021-01-01T03:00:00+01:00,2021-01-01T04:00:00+01:00,private,3641.815,1721
4,municipality,0,0,Ukjent tilhørighet,2021-01-01T04:00:00+01:00,2021-01-01T05:00:00+01:00,private,3553.285,1721
...,...,...,...,...,...,...,...,...,...
13798389,municipality,5636,5636,Unjárga,2025-05-25T19:00:00+02:00,2025-05-25T20:00:00+02:00,private,857.770,614
13798390,municipality,5636,5636,Unjárga,2025-05-25T20:00:00+02:00,2025-05-25T21:00:00+02:00,private,905.987,614
13798391,municipality,5636,5636,Unjárga,2025-05-25T21:00:00+02:00,2025-05-25T22:00:00+02:00,private,940.450,614
13798392,municipality,5636,5636,Unjárga,2025-05-25T22:00:00+02:00,2025-05-25T23:00:00+02:00,private,928.957,614


In [ ]:
df_collect_business_consump=pd.read_csv('municipalities_group_business_consumption.csv')
df_collect_business_consump

In [ ]:
df_collect_business_consump = df_collect_private_consump.drop(columns=['lastUpdatedTime', 'nameNo']) # Drop unnecessary columns

In [ ]:

df_collect_business_consump

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
0,municipality,0,0,Ukjent tilhørighet,2021-01-01T00:00:00+01:00,2021-01-01T01:00:00+01:00,business,1839.879,251
1,municipality,0,0,Ukjent tilhørighet,2021-01-01T01:00:00+01:00,2021-01-01T02:00:00+01:00,business,1813.372,251
2,municipality,0,0,Ukjent tilhørighet,2021-01-01T02:00:00+01:00,2021-01-01T03:00:00+01:00,business,1892.385,251
3,municipality,0,0,Ukjent tilhørighet,2021-01-01T03:00:00+01:00,2021-01-01T04:00:00+01:00,business,1912.907,251
4,municipality,0,0,Ukjent tilhørighet,2021-01-01T04:00:00+01:00,2021-01-01T05:00:00+01:00,business,1922.656,251
...,...,...,...,...,...,...,...,...,...
13798389,municipality,5636,5636,Unjárga,2025-05-25T19:00:00+02:00,2025-05-25T20:00:00+02:00,business,637.193,110
13798390,municipality,5636,5636,Unjárga,2025-05-25T20:00:00+02:00,2025-05-25T21:00:00+02:00,business,677.898,110
13798391,municipality,5636,5636,Unjárga,2025-05-25T21:00:00+02:00,2025-05-25T22:00:00+02:00,business,651.606,110
13798392,municipality,5636,5636,Unjárga,2025-05-25T22:00:00+02:00,2025-05-25T23:00:00+02:00,business,602.574,110


In [ ]:
df_collect_industry_consump=pd.read_csv('municipalities_group_industry_consumption.csv')
df_collect_industry_consump

In [ ]:
df_collect_industry_consump = df_collect_private_consump.drop(columns=['lastUpdatedTime', 'nameNo']) # Drop unnecessary columns

In [ ]:
df_collect_industry_consump

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
0,municipality,0,0,Ukjent tilhørighet,2021-01-01T00:00:00+01:00,2021-01-01T01:00:00+01:00,industry,11634.323,130
1,municipality,0,0,Ukjent tilhørighet,2021-01-01T01:00:00+01:00,2021-01-01T02:00:00+01:00,industry,11581.860,130
2,municipality,0,0,Ukjent tilhørighet,2021-01-01T02:00:00+01:00,2021-01-01T03:00:00+01:00,industry,11737.925,130
3,municipality,0,0,Ukjent tilhørighet,2021-01-01T03:00:00+01:00,2021-01-01T04:00:00+01:00,industry,12749.224,130
4,municipality,0,0,Ukjent tilhørighet,2021-01-01T04:00:00+01:00,2021-01-01T05:00:00+01:00,industry,12560.854,130
...,...,...,...,...,...,...,...,...,...
13789797,municipality,5636,5636,Unjárga,2025-05-24T19:00:00+02:00,2025-05-24T20:00:00+02:00,industry,92.598,19
13789798,municipality,5636,5636,Unjárga,2025-05-24T20:00:00+02:00,2025-05-24T21:00:00+02:00,industry,92.398,19
13789799,municipality,5636,5636,Unjárga,2025-05-24T21:00:00+02:00,2025-05-24T22:00:00+02:00,industry,93.325,19
13789800,municipality,5636,5636,Unjárga,2025-05-24T22:00:00+02:00,2025-05-24T23:00:00+02:00,industry,95.736,19


In [ ]:
df_collect_private_consump.name.unique() # Get unique names in the 'name' column of the private consumption DataFrame

array(['Ukjent tilhørighet', 'Oslo', 'Eigersund', 'Stavanger',
       'Haugesund', 'Sandnes', 'Sokndal', 'Lund', 'Bjerkreim', 'Hå',
       'Klepp', 'Time', 'Gjesdal', 'Sola', 'Randaberg', 'Strand',
       'Hjelmeland', 'Suldal', 'Sauda', 'Kvitsøy', 'Bokn', 'Tysvær',
       'Karmøy', 'Utsira', 'Vindafjord', 'Kristiansund', 'Molde',
       'Ålesund', 'Vanylven', 'Sande', 'Herøy', 'Ulstein', 'Hareid',
       'Ørsta', 'Stranda', 'Sykkylven', 'Sula', 'Giske', 'Vestnes',
       'Rauma', 'Aukra', 'Averøy', 'Gjemnes', 'Tingvoll', 'Sunndal',
       'Surnadal', 'Smøla', 'Aure', 'Volda', 'Fjord', 'Hustadvika',
       'Haram', 'Bodø', 'Narvik', 'Bindal', 'Sømna', 'Brønnøy', 'Vega',
       'Vevelstad', 'Alstahaug', 'Leirfjord', 'Vefsn', 'Grane',
       'Aarborte', 'Dønna', 'Nesna', 'Hemnes', 'Rana', 'Lurøy', 'Træna',
       'Rødøy', 'Meløy', 'Gildeskål', 'Beiarn', 'Saltdal', 'Fauske',
       'Sørfold', 'Steigen', 'Lødingen', 'Evenes', 'Røst', 'Værøy',
       'Flakstad', 'Vestvågøy', 'Vågan', 'Hadse

In [ ]:
consump_private_alesund=df_collect_private_consump[df_collect_private_consump['name']=='Ålesund'] # Filter data for Ålesund municipality
consump_private_alesund 

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
648,municipality,1508,1508,Ålesund,2021-01-01T00:00:00+01:00,2021-01-01T01:00:00+01:00,private,58370.730,27060
649,municipality,1508,1508,Ålesund,2021-01-01T01:00:00+01:00,2021-01-01T02:00:00+01:00,private,57961.562,27060
650,municipality,1508,1508,Ålesund,2021-01-01T02:00:00+01:00,2021-01-01T03:00:00+01:00,private,56258.645,27060
651,municipality,1508,1508,Ålesund,2021-01-01T03:00:00+01:00,2021-01-01T04:00:00+01:00,private,54556.203,27060
652,municipality,1508,1508,Ålesund,2021-01-01T04:00:00+01:00,2021-01-01T05:00:00+01:00,private,53356.273,27060
...,...,...,...,...,...,...,...,...,...
13790469,municipality,1508,1508,Ålesund,2025-05-25T19:00:00+02:00,2025-05-25T20:00:00+02:00,private,45866.402,29305
13790470,municipality,1508,1508,Ålesund,2025-05-25T20:00:00+02:00,2025-05-25T21:00:00+02:00,private,47126.195,29305
13790471,municipality,1508,1508,Ålesund,2025-05-25T21:00:00+02:00,2025-05-25T22:00:00+02:00,private,46556.260,29305
13790472,municipality,1508,1508,Ålesund,2025-05-25T22:00:00+02:00,2025-05-25T23:00:00+02:00,private,45021.470,29305


In [ ]:
consump_industry_alesund=df_collect_industry_consump[df_collect_industry_consump['name']=='Ålesund'] # Filter data for Ålesund municipality
consump_industry_alesund 

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
648,municipality,1508,1508,Ålesund,2021-01-01T00:00:00+01:00,2021-01-01T01:00:00+01:00,industry,13259.716,1029
649,municipality,1508,1508,Ålesund,2021-01-01T01:00:00+01:00,2021-01-01T02:00:00+01:00,industry,13558.855,1029
650,municipality,1508,1508,Ålesund,2021-01-01T02:00:00+01:00,2021-01-01T03:00:00+01:00,industry,13649.810,1029
651,municipality,1508,1508,Ålesund,2021-01-01T03:00:00+01:00,2021-01-01T04:00:00+01:00,industry,13533.480,1029
652,municipality,1508,1508,Ålesund,2021-01-01T04:00:00+01:00,2021-01-01T05:00:00+01:00,industry,13484.362,1029
...,...,...,...,...,...,...,...,...,...
13781877,municipality,1508,1508,Ålesund,2025-05-24T19:00:00+02:00,2025-05-24T20:00:00+02:00,industry,18226.408,842
13781878,municipality,1508,1508,Ålesund,2025-05-24T20:00:00+02:00,2025-05-24T21:00:00+02:00,industry,17460.377,842
13781879,municipality,1508,1508,Ålesund,2025-05-24T21:00:00+02:00,2025-05-24T22:00:00+02:00,industry,17923.867,842
13781880,municipality,1508,1508,Ålesund,2025-05-24T22:00:00+02:00,2025-05-24T23:00:00+02:00,industry,17545.988,842


In [ ]:
consump_business_alesund=df_collect_business_consump[df_collect_business_consump['name']=='Ålesund'] # Filter data for Ålesund municipality
consump_business_alesund 

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
648,municipality,1508,1508,Ålesund,2021-01-01T00:00:00+01:00,2021-01-01T01:00:00+01:00,business,30526.664,3784
649,municipality,1508,1508,Ålesund,2021-01-01T01:00:00+01:00,2021-01-01T02:00:00+01:00,business,30582.870,3784
650,municipality,1508,1508,Ålesund,2021-01-01T02:00:00+01:00,2021-01-01T03:00:00+01:00,business,30352.254,3784
651,municipality,1508,1508,Ålesund,2021-01-01T03:00:00+01:00,2021-01-01T04:00:00+01:00,business,30774.594,3784
652,municipality,1508,1508,Ålesund,2021-01-01T04:00:00+01:00,2021-01-01T05:00:00+01:00,business,30846.584,3784
...,...,...,...,...,...,...,...,...,...
13790469,municipality,1508,1508,Ålesund,2025-05-25T19:00:00+02:00,2025-05-25T20:00:00+02:00,business,26713.293,3373
13790470,municipality,1508,1508,Ålesund,2025-05-25T20:00:00+02:00,2025-05-25T21:00:00+02:00,business,26257.412,3373
13790471,municipality,1508,1508,Ålesund,2025-05-25T21:00:00+02:00,2025-05-25T22:00:00+02:00,business,26194.588,3373
13790472,municipality,1508,1508,Ålesund,2025-05-25T22:00:00+02:00,2025-05-25T23:00:00+02:00,business,25590.326,3373


In [ ]:
consump_private_alesund.loc[:, 'startTime'] = pd.to_datetime(consump_private_alesund.loc[:, 'startTime'], utc=True) # Convert startTime to datetime with UTC bcause the API returns it + time zone
consump_private_alesund.loc[:, 'endTime'] = pd.to_datetime(consump_private_alesund.loc[:, 'endTime'], utc=True) 
consump_private_alesund 

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
648,municipality,1508,1508,Ålesund,2020-12-31 23:00:00+00:00,2021-01-01 00:00:00+00:00,private,58370.730,27060
649,municipality,1508,1508,Ålesund,2021-01-01 00:00:00+00:00,2021-01-01 01:00:00+00:00,private,57961.562,27060
650,municipality,1508,1508,Ålesund,2021-01-01 01:00:00+00:00,2021-01-01 02:00:00+00:00,private,56258.645,27060
651,municipality,1508,1508,Ålesund,2021-01-01 02:00:00+00:00,2021-01-01 03:00:00+00:00,private,54556.203,27060
652,municipality,1508,1508,Ålesund,2021-01-01 03:00:00+00:00,2021-01-01 04:00:00+00:00,private,53356.273,27060
...,...,...,...,...,...,...,...,...,...
13790469,municipality,1508,1508,Ålesund,2025-05-25 17:00:00+00:00,2025-05-25 18:00:00+00:00,private,45866.402,29305
13790470,municipality,1508,1508,Ålesund,2025-05-25 18:00:00+00:00,2025-05-25 19:00:00+00:00,private,47126.195,29305
13790471,municipality,1508,1508,Ålesund,2025-05-25 19:00:00+00:00,2025-05-25 20:00:00+00:00,private,46556.260,29305
13790472,municipality,1508,1508,Ålesund,2025-05-25 20:00:00+00:00,2025-05-25 21:00:00+00:00,private,45021.470,29305


In [ ]:
nan_par_colonne = consump_private_alesund.isna().sum()  # Count NaN values in each column
print(nan_par_colonne)


type                  0
id                    0
municipalityNumber    0
name                  0
startTime             0
endTime               0
consumptionGroup      0
quantityKwh           0
meteringPointCount    0
dtype: int64


In [ ]:
nbr_doublons = consump_private_alesund['startTime'].duplicated().sum() # Count the number of duplicate entries in the 'startTime' column
print("Nombre de doublons dans startTime :", nbr_doublons)


Nombre de doublons dans startTime : 0


In [66]:
consump_business_alesund.loc[:, 'startTime'] = pd.to_datetime(consump_business_alesund.loc[:, 'startTime'], utc=True)
consump_business_alesund.loc[:, 'endTime'] = pd.to_datetime(consump_business_alesund.loc[:, 'endTime'], utc=True)
consump_business_alesund

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
648,municipality,1508,1508,Ålesund,2020-12-31 23:00:00+00:00,2021-01-01 00:00:00+00:00,business,30526.664,3784
649,municipality,1508,1508,Ålesund,2021-01-01 00:00:00+00:00,2021-01-01 01:00:00+00:00,business,30582.870,3784
650,municipality,1508,1508,Ålesund,2021-01-01 01:00:00+00:00,2021-01-01 02:00:00+00:00,business,30352.254,3784
651,municipality,1508,1508,Ålesund,2021-01-01 02:00:00+00:00,2021-01-01 03:00:00+00:00,business,30774.594,3784
652,municipality,1508,1508,Ålesund,2021-01-01 03:00:00+00:00,2021-01-01 04:00:00+00:00,business,30846.584,3784
...,...,...,...,...,...,...,...,...,...
13790469,municipality,1508,1508,Ålesund,2025-05-25 17:00:00+00:00,2025-05-25 18:00:00+00:00,business,26713.293,3373
13790470,municipality,1508,1508,Ålesund,2025-05-25 18:00:00+00:00,2025-05-25 19:00:00+00:00,business,26257.412,3373
13790471,municipality,1508,1508,Ålesund,2025-05-25 19:00:00+00:00,2025-05-25 20:00:00+00:00,business,26194.588,3373
13790472,municipality,1508,1508,Ålesund,2025-05-25 20:00:00+00:00,2025-05-25 21:00:00+00:00,business,25590.326,3373


In [67]:
nan_par_colonne = consump_business_alesund.isna().sum()
print(nan_par_colonne)

type                  0
id                    0
municipalityNumber    0
name                  0
startTime             0
endTime               0
consumptionGroup      0
quantityKwh           0
meteringPointCount    0
dtype: int64


In [ ]:
nbr_doublons = consump_business_alesund['startTime'].duplicated().sum()  # Count the number of duplicate entries in the 'startTime' column
print("Nombre de doublons dans startTime :", nbr_doublons)

Nombre de doublons dans startTime : 0


In [69]:
consump_industry_alesund.loc[:, 'startTime'] = pd.to_datetime(consump_industry_alesund.loc[:, 'startTime'], utc=True)
consump_industry_alesund.loc[:, 'endTime'] = pd.to_datetime(consump_industry_alesund.loc[:, 'endTime'], utc=True)
consump_industry_alesund

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
648,municipality,1508,1508,Ålesund,2020-12-31 23:00:00+00:00,2021-01-01 00:00:00+00:00,industry,13259.716,1029
649,municipality,1508,1508,Ålesund,2021-01-01 00:00:00+00:00,2021-01-01 01:00:00+00:00,industry,13558.855,1029
650,municipality,1508,1508,Ålesund,2021-01-01 01:00:00+00:00,2021-01-01 02:00:00+00:00,industry,13649.810,1029
651,municipality,1508,1508,Ålesund,2021-01-01 02:00:00+00:00,2021-01-01 03:00:00+00:00,industry,13533.480,1029
652,municipality,1508,1508,Ålesund,2021-01-01 03:00:00+00:00,2021-01-01 04:00:00+00:00,industry,13484.362,1029
...,...,...,...,...,...,...,...,...,...
13781877,municipality,1508,1508,Ålesund,2025-05-24 17:00:00+00:00,2025-05-24 18:00:00+00:00,industry,18226.408,842
13781878,municipality,1508,1508,Ålesund,2025-05-24 18:00:00+00:00,2025-05-24 19:00:00+00:00,industry,17460.377,842
13781879,municipality,1508,1508,Ålesund,2025-05-24 19:00:00+00:00,2025-05-24 20:00:00+00:00,industry,17923.867,842
13781880,municipality,1508,1508,Ålesund,2025-05-24 20:00:00+00:00,2025-05-24 21:00:00+00:00,industry,17545.988,842


In [70]:
nan_par_colonne = consump_industry_alesund.isna().sum()
print(nan_par_colonne)

type                  0
id                    0
municipalityNumber    0
name                  0
startTime             0
endTime               0
consumptionGroup      0
quantityKwh           0
meteringPointCount    0
dtype: int64


In [ ]:
nbr_doublons = consump_industry_alesund['startTime'].duplicated().sum() 
print("Nombre de doublons dans startTime :", nbr_doublons)

Nombre de doublons dans startTime : 0


In [ ]:

# Extraire les dates de chaque groupe 
dates1 = set(consump_business_alesund['startTime'])
dates2 = set(consump_industry_alesund['startTime'])
dates3 = set(consump_private_alesund['startTime'])

# Intersection des dates
dates_communes = dates1 & dates2 & dates3
groupe1_filtré = consump_private_alesund[consump_private_alesund['startTime'].isin(dates_communes)]
groupe2_filtré = consump_business_alesund[consump_business_alesund['startTime'].isin(dates_communes)]
groupe3_filtré = consump_industry_alesund[consump_industry_alesund['startTime'].isin(dates_communes)]
# Concaténation des DataFrames filtrés
resultat = pd.concat([groupe1_filtré, groupe2_filtré, groupe3_filtré])
resultat = resultat.sort_values(by='startTime')
resultat

,type,id,municipalityNumber,name,startTime,endTime,consumptionGroup,quantityKwh,meteringPointCount
648,municipality,1508,1508,Ålesund,2020-12-31 23:00:00+00:00,2021-01-01 00:00:00+00:00,private,58370.730,27060
648,municipality,1508,1508,Ålesund,2020-12-31 23:00:00+00:00,2021-01-01 00:00:00+00:00,industry,13259.716,1029
648,municipality,1508,1508,Ålesund,2020-12-31 23:00:00+00:00,2021-01-01 00:00:00+00:00,business,30526.664,3784
649,municipality,1508,1508,Ålesund,2021-01-01 00:00:00+00:00,2021-01-01 01:00:00+00:00,private,57961.562,27060
649,municipality,1508,1508,Ålesund,2021-01-01 00:00:00+00:00,2021-01-01 01:00:00+00:00,industry,13558.855,1029
...,...,...,...,...,...,...,...,...,...
13781880,municipality,1508,1508,Ålesund,2025-05-24 20:00:00+00:00,2025-05-24 21:00:00+00:00,private,47385.406,29305
13781880,municipality,1508,1508,Ålesund,2025-05-24 20:00:00+00:00,2025-05-24 21:00:00+00:00,business,24504.025,3373
13781881,municipality,1508,1508,Ålesund,2025-05-24 21:00:00+00:00,2025-05-24 22:00:00+00:00,business,25000.594,3373
13781881,municipality,1508,1508,Ålesund,2025-05-24 21:00:00+00:00,2025-05-24 22:00:00+00:00,private,46403.700,29305


In [ ]:
resultat.to_csv("consommation_Alesund_per_group.csv", index=False) # Save the result to a CSV file

### Extraction des groupes de consommation
### Contexte et objectif
l'extraction des groupes de consommation disponibles via l'API Elhub. Cette étape est cruciale pour comprendre quels types de consommateurs existent dans le système avant de procéder à l'extraction des données détaillées.

Avantages :

Adaptabilité : Fonctionne même si l'API ajoute de nouveaux champs

Maintenabilité : Aucune modification de code nécessaire pour de nouveaux attributs

In [ ]:


headers = {
    
} # Make a request to the Elhub API to get consumption groups data

response = requests.get('https://api.elhub.no/energy-data/v0/consumption-groups', headers=headers)  # Check if the request was successful
if response.status_code == 200: # Check if the request was successful
    json_data = response.json()
    data_list = json_data.get("data", [])

    rows = []
    all_keys = set()

    # Parcours des objets JSON pour extraire toutes les clés
    for item in data_list:
        flat_item = {}
        for key, value in item.items():
            if isinstance(value, dict): 
                for sub_key, sub_value in value.items(): # Check if the value is a dictionary
                    flat_item[sub_key] = sub_value
                    all_keys.add(sub_key)
            else:
                flat_item[key] = value
                all_keys.add(key)
        rows.append(flat_item)

    # Génération CSV avec les en-têtes détectées dynamiquement
    if rows:
        with open("groupes_de_consommation.csv", mode="w", newline='', encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=sorted(all_keys))
            writer.writeheader()
            writer.writerows(rows)
        print(" Données exportées dans groupes_de_consommation.csv")
    else:
        print(" Aucune donnée trouvée.")
else:
    print(" Échec de l'appel API :", response.status_code) # Print error if the API call was not successful

 Données exportées dans groupes_de_consommation.csv


In [ ]:
df_collect_groupe=pd.read_csv('groupes_de_consommation.csv')
df_collect_groupe

,description,id,name,type
0,Meteringpoints that either have NACE Code XX (...,household,Household,consumption-groups
1,Cabins and vacation homes nace XY,cabin,Cabins and holiday properties,consumption-groups
2,"All agriculture, forestry, fishing incl green ...",primary,Primary Industry,consumption-groups
3,All industry section codes B through F,secondary,Secondary Industry,consumption-groups
4,All section codes G through U. Also includes a...,tertiary,Tertiary Industry,consumption-groups
5,Combination of Primary and Secondary Industry,industry,Industry,consumption-groups
6,Combination of Household and Cabin,private,Private,consumption-groups
7,Same as Tertiary Industry,business,Business,consumption-groups
8,*,*,*,consumption-groups


Ce code configure les paramètres nécessaires pour collecter des données de consommation énergétique via l'API Elhub

Type de données : Consommation par groupe MBA par heure

Format : Respecter la casse exacte exigée par l'API

Description : Collecte les données de consommation électrique agrégées par groupes de zones de prix (MBA - Market Balance Area)

Authentification : Vérifiez que vous disposez des credentials appropriés pour accéder à l'API Elhub

Limites de l'API : Respectez les quotas et limites de taux imposés par Elhub

Format des données : Les données seront structurées selon le schéma MBA (Market Balance Area)

Mise à jour : La date de fin est dynamique et correspond toujours au moment d'exécution

In [ ]:

# Constantes
dataset = "CONSUMPTION_PER_GROUP_MBA_HOUR"  # Respecter la casse exacte exigée par l'API
base_url = "https://api.elhub.no/energy-data/v0/price-areas" # Base URL for the API endpoint

# Dates de début et de fin
start_date = datetime(2020, 1, 1)  # Date de début pour la collecte de données autorisée par l'API
end_limit = datetime.now()

# Fichier de sortie
file_name = "consumption_price_areas.csv"

# Initialiser les collections pour stocker toutes les données
all_rows = []              # Liste pour stocker toutes les lignes de données  
all_fieldnames = set()                       # Ensemble pour collecter automatiquement tous les en-têtes possibles

# Boucle principale pour traiter chaque jour
current_date = start_date      # Initialize current date to start date will be incremented day by day    
while current_date < end_limit: 
    end_date = current_date + timedelta(days=1)            

    # Format des dates pour l'URL
    start_str = current_date.strftime("%Y-%m-%d")
    end_str = end_date.strftime("%Y-%m-%d")

    # Construction de l'URL API
    url = (
        f"{base_url}?dataset={dataset}"
        f"&startDate={start_str}"
        f"&endDate={end_str}"
    )

    print(f"Appel API : {start_str} → {end_str}")

    try:
        # Effectuer la requête API
        response = requests.get(url)
        
        if response.status_code == 200:
            json_full = response.json()
            data = json_full.get("data", [])
            
            # Traiter chaque élément de données
            for item in data:
                if not isinstance(item, dict):
                    continue
                
                # Structure de base de l'item
                base_row = {
                    "type": item.get("type"), 
                    "id": item.get("id")
                }
                
                attr = item.get("attributes", {})    # Extraire le champ "attributes", par défaut un dictionnaire vide si non présent
                
                # Ajouter automatiquement tous les attributs au niveau municipality 
                for attr_key, attr_value in attr.items():
                    if attr_key != "consumptionPerGroupMbaHour":
                        base_row[attr_key] = attr_value 
                
                # Traiter les données de consommation horaire
                for record in attr.get("consumptionPerGroupMbaHour", []):
                    # Créer une ligne complète en combinant les infos de base et les données horaires
                    row = {**base_row}  # Copier les données de base
                    
                    # Ajouter automatiquement toutes les données du record
                    for record_key, record_value in record.items():
                        row[record_key] = record_value
                    
                    # Ajouter toutes les clés trouvées à notre ensemble d'en-têtes
                    all_fieldnames.update(row.keys())
                    all_rows.append(row)
            
            if data:
                print(f"  {len(data)} records processed for date {start_str}")
            else:
                print(f"  No data for date {start_str}")
                
        else:
            print(f"API call failed with status code: {response.status_code}")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"API error: {e}")
    
    # Passer au jour suivant
    current_date = end_date       # Increment the current date by one day to process the next date
    time.sleep(1)  # Pause pour éviter les limitations de taux

# Écrire toutes les données dans le fichier CSV
if all_rows:
    with open(file_name, mode="w", newline="", encoding="utf-8") as file:
        # Trier les en-têtes pour avoir un ordre cohérent
        sorted_fieldnames = sorted(all_fieldnames)
        writer = csv.DictWriter(file, fieldnames=sorted_fieldnames)
        writer.writeheader()
        writer.writerows(all_rows)
    
    print(f"\nData saved to file: {file_name}")
    print(f"Detected fieldnames: {sorted_fieldnames}")
    print(f"Total rows: {len(all_rows)}")
else:
    print("No data found in the specified date range")

 Appel API : 2020-01-01 → 2020-01-02
 Aucune donnée pour 2020-01-01
 Appel API : 2020-01-02 → 2020-01-03
 Aucune donnée pour 2020-01-02
 Appel API : 2020-01-03 → 2020-01-04
 Aucune donnée pour 2020-01-03
 Appel API : 2020-01-04 → 2020-01-05
 Aucune donnée pour 2020-01-04
 Appel API : 2020-01-05 → 2020-01-06
 Aucune donnée pour 2020-01-05
 Appel API : 2020-01-06 → 2020-01-07
 Aucune donnée pour 2020-01-06
 Appel API : 2020-01-07 → 2020-01-08
 Aucune donnée pour 2020-01-07
 Appel API : 2020-01-08 → 2020-01-09
 Aucune donnée pour 2020-01-08
 Appel API : 2020-01-09 → 2020-01-10
 Aucune donnée pour 2020-01-09
 Appel API : 2020-01-10 → 2020-01-11
 Aucune donnée pour 2020-01-10
 Appel API : 2020-01-11 → 2020-01-12
 Aucune donnée pour 2020-01-11
 Appel API : 2020-01-12 → 2020-01-13
 Aucune donnée pour 2020-01-12
 Appel API : 2020-01-13 → 2020-01-14
 Aucune donnée pour 2020-01-13
 Appel API : 2020-01-14 → 2020-01-15
 Aucune donnée pour 2020-01-14
 Appel API : 2020-01-15 → 2020-01-16
 Aucune don

In [25]:
df_cons_collect=pd.read_csv('consumption_price_areas.csv') # Read the collected consumption data from the CSV file 

In [26]:
df_cons_collect

,consumptionGroup,country,eic,endTime,id,lastUpdatedTime,meteringPointCount,name,priceArea,quantityKwh,startTime,type
0,cabin,NO,10YNO-1--------2,2025-07-15T01:00:00+02:00,NO1,2025-07-17T17:55:27+02:00,109677,NO1,NO1,44426.387,2025-07-15T00:00:00+02:00,price-areas
1,cabin,NO,10YNO-1--------2,2025-07-15T02:00:00+02:00,NO1,2025-07-17T17:55:27+02:00,109677,NO1,NO1,41092.910,2025-07-15T01:00:00+02:00,price-areas
2,cabin,NO,10YNO-1--------2,2025-07-15T03:00:00+02:00,NO1,2025-07-17T17:55:27+02:00,109677,NO1,NO1,38442.156,2025-07-15T02:00:00+02:00,price-areas
3,cabin,NO,10YNO-1--------2,2025-07-15T04:00:00+02:00,NO1,2025-07-17T17:55:27+02:00,109677,NO1,NO1,36801.008,2025-07-15T03:00:00+02:00,price-areas
4,cabin,NO,10YNO-1--------2,2025-07-15T05:00:00+02:00,NO1,2025-07-17T17:55:27+02:00,109677,NO1,NO1,35028.797,2025-07-15T04:00:00+02:00,price-areas
...,...,...,...,...,...,...,...,...,...,...,...,...
1195,tertiary,NO,10Y1001A1001A48H,2025-07-16T20:00:00+02:00,NO5,2025-07-17T18:11:07+02:00,29205,NO5,NO5,437107.250,2025-07-16T19:00:00+02:00,price-areas
1196,tertiary,NO,10Y1001A1001A48H,2025-07-16T21:00:00+02:00,NO5,2025-07-17T18:11:07+02:00,29205,NO5,NO5,430533.030,2025-07-16T20:00:00+02:00,price-areas
1197,tertiary,NO,10Y1001A1001A48H,2025-07-16T22:00:00+02:00,NO5,2025-07-17T18:11:07+02:00,29205,NO5,NO5,422526.120,2025-07-16T21:00:00+02:00,price-areas
1198,tertiary,NO,10Y1001A1001A48H,2025-07-16T23:00:00+02:00,NO5,2025-07-17T18:11:07+02:00,29205,NO5,NO5,408009.560,2025-07-16T22:00:00+02:00,price-areas


In [27]:
df_cons_collect = df_cons_collect.drop(columns=['lastUpdatedTime']) # Drop unnecessary columns

In [28]:
df_cons_collect

,consumptionGroup,country,eic,endTime,id,meteringPointCount,name,priceArea,quantityKwh,startTime,type
0,cabin,NO,10YNO-1--------2,2025-07-15T01:00:00+02:00,NO1,109677,NO1,NO1,44426.387,2025-07-15T00:00:00+02:00,price-areas
1,cabin,NO,10YNO-1--------2,2025-07-15T02:00:00+02:00,NO1,109677,NO1,NO1,41092.910,2025-07-15T01:00:00+02:00,price-areas
2,cabin,NO,10YNO-1--------2,2025-07-15T03:00:00+02:00,NO1,109677,NO1,NO1,38442.156,2025-07-15T02:00:00+02:00,price-areas
3,cabin,NO,10YNO-1--------2,2025-07-15T04:00:00+02:00,NO1,109677,NO1,NO1,36801.008,2025-07-15T03:00:00+02:00,price-areas
4,cabin,NO,10YNO-1--------2,2025-07-15T05:00:00+02:00,NO1,109677,NO1,NO1,35028.797,2025-07-15T04:00:00+02:00,price-areas
...,...,...,...,...,...,...,...,...,...,...,...
1195,tertiary,NO,10Y1001A1001A48H,2025-07-16T20:00:00+02:00,NO5,29205,NO5,NO5,437107.250,2025-07-16T19:00:00+02:00,price-areas
1196,tertiary,NO,10Y1001A1001A48H,2025-07-16T21:00:00+02:00,NO5,29205,NO5,NO5,430533.030,2025-07-16T20:00:00+02:00,price-areas
1197,tertiary,NO,10Y1001A1001A48H,2025-07-16T22:00:00+02:00,NO5,29205,NO5,NO5,422526.120,2025-07-16T21:00:00+02:00,price-areas
1198,tertiary,NO,10Y1001A1001A48H,2025-07-16T23:00:00+02:00,NO5,29205,NO5,NO5,408009.560,2025-07-16T22:00:00+02:00,price-areas


In [ ]:
df_cons_collect['consumptionGroup'].unique()     # Get unique consumption groups from the DataFrame

array(['cabin', 'household', 'primary', 'secondary', 'tertiary'],
      dtype=object)

In [ ]:
df_cons_collect.to_csv("consumption_price_areas.csv", index=False) # Save the result to a CSV file

### Vue d'ensemble

Ce code effectue un test initial de l'API Elhub pour vérifier la connectivité et comprendre la structure des données avant d'implémenter l'automatisation complète.

### Endpoint utilisé

URL : https://api.elhub.no/energy-data/v0/municipalities

Scope : Données au niveau des municipalités

### Paramètres de requête

dataset : PRODUCTION_PER_METERING_POINT_TYPE_MUNICIPALITY_HOUR

Type de données : Production par type de point de mesure par municipalité et par heure

### Objectifs du test

Vérification de connectivité : S'assurer que l'API répond correctement

Exploration des données : Comprendre la structure JSON retournée

Validation des paramètres : Tester le format des dates et paramètres

Gestion d'erreurs : Identifier les codes d'erreur possibles

### Résultats attendus

#### Succès (status_code = 200)

Affichage de la réponse JSON complète

#### Échec (status_code ≠ 200)

Codes d'erreur possibles :

401 : Authentification requise

403 : Accès interdit

404 : Endpoint non trouvé

422 : Paramètres invalides

429 : Trop de requêtes (rate limiting)

In [ ]:

headers = {
    
}

response = requests.get('https://api.elhub.no/energy-data/v0/municipalities?dataset=PRODUCTION_PER_METERING_POINT_TYPE_MUNICIPALITY_HOUR&startDate=2023-05-03T20:00:00%2B02:00&endDate=2023-05-04T00:00:00%2B02:00', headers=headers)
if response.status_code == 200:
    print(response.json())
else:
    print('Error:', response.status_code)

{'meta': {'created': '2025-05-19T15:16:27+02:00', 'lastUpdated': '2025-03-29T01:45:22+01:00'}, 'links': {'self': 'https://api.elhub.no/energy-data/v0/municipalities?dataset=PRODUCTION_PER_METERING_POINT_TYPE_MUNICIPALITY_HOUR&endDate=2023-05-04T00%3A00%3A00%2B02%3A00&startDate=2023-05-03T20%3A00%3A00%2B02%3A00'}, 'data': [{'type': 'municipality', 'id': '*', 'attributes': {'municipalityNumber': '*', 'name': '*', 'nameNo': '*', 'productionPerMeteringPointTypeMunicipalityHour': []}}, {'type': 'municipality', 'id': '0000', 'attributes': {'municipalityNumber': '0000', 'name': 'Ukjent tilhørighet', 'nameNo': 'Ukjent tilhørighet', 'productionPerMeteringPointTypeMunicipalityHour': []}}, {'type': 'municipality', 'id': '0301', 'attributes': {'municipalityNumber': '0301', 'name': 'Oslo', 'nameNo': 'Oslo', 'productionPerMeteringPointTypeMunicipalityHour': [{'startTime': '2023-05-03T20:00:00+02:00', 'endTime': '2023-05-03T21:00:00+02:00', 'meteringPointTypeCode': 'E19', 'quantityKwh': 6.609, 'lastU

Metering Point Type
Codes according to ebIX:

E17 Consumption
E18 Production
E19 Combined
E20 Exchange

In [ ]:

# Constants
dataset = "PRODUCTION_PER_METERING_POINT_TYPE_MUNICIPALITY_HOUR"  # Dataset identifier in the API
base_url = "https://api.elhub.no/energy-data/v0/municipalities"    # Base URL for the API endpoint

# Start and end dates
start_date = datetime(2025, 7, 15)   # Start date for data collection authorized by the API
end_limit = datetime.now()        # Current date and time

# Output CSV file
file_name = "municipalities_production.csv"

# Collections pour stocker toutes les données et détecter les colonnes
all_rows = []
all_fieldnames = set()

# Boucle principale pour traiter chaque jour
current_date = start_date
while current_date < end_limit:
    # Define the date window (1 day per request)
    end_date = current_date + timedelta(days=1)
    
    # Format dates as ISO 8601 (YYYY-MM-DD)
    start_str = current_date.strftime("%Y-%m-%d")
    end_str = end_date.strftime("%Y-%m-%d")
    
    # Construct the API URL
    url = (
        f"{base_url}?dataset={dataset}"
        f"&startDate={start_str}"
        f"&endDate={end_str}"
    )
    
    print(f"API call: {start_str} → {end_str}")    # Print the API call being made
    
    try:   # Make the API request in a try-except block to handle potential errors
        response = requests.get(url)      # Send GET request to the API
        
        # Check HTTP response
        if response.status_code == 200:    # Check if the request was successful (200 means OK)
            json_full = response.json()                # Parse the JSON response from the API
            data = json_full.get("data", [])          # Extract the "data" field from the JSON response, defaulting to an empty list if not present
            
            # Loop over each municipality record
            for item in data:
                if not isinstance(item, dict):
                    continue
                
                # Structure de base avec les champs de niveau supérieur
                base_row = {
                    "type": item.get("type"),
                    "id": item.get("id")
                }
                
                attr = item.get("attributes", {})
                
                # Ajouter automatiquement tous les attributs (sauf les données imbriquées)
                for attr_key, attr_value in attr.items():  # Loop through each attribute in the attributes dictionary
                    if attr_key != "productionPerMeteringPointTypeMunicipalityHour":      # Skip the nested production data 
                        base_row[attr_key] = attr_value  # Add each attribute to the base row
                
                # Loop over each hourly production record
                for record in attr.get("productionPerMeteringPointTypeMunicipalityHour", []):     # Extract the hourly production data
                    # Créer une ligne complète en combinant les infos de base et les données horaires
                    row = {**base_row}  # Copier les données de base
                    
                    # Ajouter automatiquement toutes les données du record
                    for record_key, record_value in record.items():
                        row[record_key] = record_value
                    
                    # Ajouter toutes les clés trouvées à notre ensemble d'en-têtes
                    all_fieldnames.update(row.keys())
                    all_rows.append(row)
            
            if data: # Print the number of records processed for the current date
                print(f"  {len(data)} municipality records processed for {start_str}")
            else:
                print(f"  No data available for {start_str}")  # Print if no data was found for the current date
                
        else:   # Print error if the API call was not successful
            print(f"API call failed: HTTP {response.status_code}")
            print(f"Response: {response.text}")
            
    except Exception as e: # Handle any exceptions that occur during the API call
        print(f"API error: {e}")    # Print any exceptions that occur during the API call
    
    # Move to the next day
    current_date = end_date   # Increment the current date by one day to process the next date
    time.sleep(1)  # Prevent rate limiting (429 errors)

# Écrire toutes les données dans le fichier CSV avec les en-têtes détectées automatiquement
if all_rows:   # Check if there are any rows to write
    with open(file_name, mode="w", newline="", encoding="utf-8") as file:  # Open the file for writing
        # Trier les en-têtes pour avoir un ordre cohérent
        sorted_fieldnames = sorted(all_fieldnames)
        writer = csv.DictWriter(file, fieldnames=sorted_fieldnames)   
        writer.writeheader()
        writer.writerows(all_rows)
    
    print(f"\nAll data saved in CSV file: {file_name}")
    print(f"Automatically detected columns: {sorted_fieldnames}")
    print(f"Total rows written: {len(all_rows)}")
else:
    print("No data found in the specified date range")

API call: 2021-01-01 → 2021-01-02
  48 records written for 2021-01-01
API call: 2021-01-02 → 2021-01-03
  456 records written for 2021-01-02
API call: 2021-01-03 → 2021-01-04
  456 records written for 2021-01-03
API call: 2021-01-04 → 2021-01-05
  456 records written for 2021-01-04
API call: 2021-01-05 → 2021-01-06
  456 records written for 2021-01-05
API call: 2021-01-06 → 2021-01-07
  432 records written for 2021-01-06
API call: 2021-01-07 → 2021-01-08
  432 records written for 2021-01-07
API call: 2021-01-08 → 2021-01-09
  432 records written for 2021-01-08
API call: 2021-01-09 → 2021-01-10
  432 records written for 2021-01-09
API call: 2021-01-10 → 2021-01-11
  432 records written for 2021-01-10
API call: 2021-01-11 → 2021-01-12
  432 records written for 2021-01-11
API call: 2021-01-12 → 2021-01-13
  432 records written for 2021-01-12
API call: 2021-01-13 → 2021-01-14
  432 records written for 2021-01-13
API call: 2021-01-14 → 2021-01-15
  432 records written for 2021-01-14
API cal

In [6]:

df_production=pd.read_csv('municipalities_production.csv')

In [4]:
df_production

,type,id,municipalityNumber,name,nameNo,startTime,endTime,meteringPointTypeCode,quantityKwh,lastUpdatedTime
0,municipality,1103,1103,Stavanger,Stavanger,2021-01-01T00:00:00+01:00,2021-01-01T01:00:00+01:00,E19,0.000,2025-03-30T11:46:56+02:00
1,municipality,1103,1103,Stavanger,Stavanger,2021-01-01T01:00:00+01:00,2021-01-01T02:00:00+01:00,E19,0.000,2025-03-30T11:46:56+02:00
2,municipality,1103,1103,Stavanger,Stavanger,2021-01-01T02:00:00+01:00,2021-01-01T03:00:00+01:00,E19,0.000,2025-03-30T11:46:56+02:00
3,municipality,1103,1103,Stavanger,Stavanger,2021-01-01T03:00:00+01:00,2021-01-01T04:00:00+01:00,E19,0.000,2025-03-30T11:46:56+02:00
4,municipality,1103,1103,Stavanger,Stavanger,2021-01-01T04:00:00+01:00,2021-01-01T05:00:00+01:00,E19,0.000,2025-03-30T11:46:56+02:00
...,...,...,...,...,...,...,...,...,...,...
9028103,municipality,5605,5605,Sør-Varanger,Sør-Varanger,2025-05-31T19:00:00+02:00,2025-05-31T20:00:00+02:00,E19,0.040,2025-06-01T17:56:32+02:00
9028104,municipality,5605,5605,Sør-Varanger,Sør-Varanger,2025-05-31T20:00:00+02:00,2025-05-31T21:00:00+02:00,E19,0.036,2025-06-01T17:56:32+02:00
9028105,municipality,5605,5605,Sør-Varanger,Sør-Varanger,2025-05-31T21:00:00+02:00,2025-05-31T22:00:00+02:00,E19,0.058,2025-06-01T17:56:32+02:00
9028106,municipality,5605,5605,Sør-Varanger,Sør-Varanger,2025-05-31T22:00:00+02:00,2025-05-31T23:00:00+02:00,E19,0.001,2025-06-01T17:56:32+02:00


## Test API Elhub - Capacité installée
### Vue d'ensemble
Ce test explore un nouveau dataset de l'API Elhub concernant la capacité installée par type de point de mesure et par groupe au niveau municipal avec une granularité quotidienne.
### Analyse du dataset
#### Paramètres du dataset

Dataset : INSTALLED_CAPACITY_PER_METERING_POINT_TYPE_GROUP_MUNICIPALITY_DAILY

Granularité : Quotidienne (DAILY)

Scope : Municipalités

Métriques : Capacité installée par type de point de mesure et par groupe
#### Paramètres de la requête
##### Période testée

Début : 2023-05-03T00:00:00+02:00 (3 mai 2023, 00h00)

Fin : 2023-05-04T00:00:00+02:00 (4 mai 2023, 00h00)

Durée : 1day
### Cas d'usage potentiels
1. Planification énergétique

Analyser la capacité installée par région

Planifier les investissements infrastructurels

2. Analyse de transition énergétique

Suivre l'évolution de la capacité renouvelable

Comparer les types de production 

In [ ]:

headers = {
    
}

response = requests.get('https://api.elhub.no/energy-data/v0/municipalities?dataset=INSTALLED_CAPACITY_PER_METERING_POINT_TYPE_GROUP_MUNICIPALITY_DAILY&startDate=2023-05-03T00:00:00%2B02:00&endDate=2023-05-04T00:00:00%2B02:00', headers=headers)
if response.status_code == 200:
    print(response.json())
else:
    print('Error:', response.status_code)

{'meta': {'created': '2025-06-02T14:03:57+02:00', 'lastUpdated': '2025-06-02T08:28:23+02:00'}, 'links': {'self': 'https://api.elhub.no/energy-data/v0/municipalities?dataset=INSTALLED_CAPACITY_PER_METERING_POINT_TYPE_GROUP_MUNICIPALITY_DAILY&endDate=2023-05-04T00%3A00%3A00%2B02%3A00&startDate=2023-05-03T20%3A00%3A00%2B02%3A00'}, 'data': [{'type': 'municipality', 'id': '4032', 'attributes': {'municipalityNumber': '4032', 'name': 'Fyresdal', 'nameNo': 'Fyresdal', 'installedCapacityPerMeteringPointTypeGroupMunicipalityDaily': [{'usageDateId': 20230503, 'municipalityId': '4032', 'meteringPointTypeCode': 'E19', 'productionGroup': 'solar', 'installedCapacity': 213.0, 'lastUpdatedTime': '2025-06-02T08:28:23+02:00'}, {'usageDateId': 20230503, 'municipalityId': '4032', 'meteringPointTypeCode': 'E18', 'productionGroup': 'hydro', 'installedCapacity': 132303.0, 'lastUpdatedTime': '2025-06-02T08:28:23+02:00'}]}}, {'type': 'municipality', 'id': '1822', 'attributes': {'municipalityNumber': '1822', 'na

In [7]:

# Constants
dataset = "INSTALLED_CAPACITY_PER_METERING_POINT_TYPE_GROUP_MUNICIPALITY_DAILY"  # Dataset identifier in the API
base_url = "https://api.elhub.no/energy-data/v0/municipalities"   # Base URL for the API endpoint

# Start and end dates
start_date = datetime(2025, 7, 15) # Start date for data collection authorized by the API
end_limit = datetime.now()

# Output CSV file
file_name = "municipalities_installed_capacity.csv"  

# Collections pour stocker toutes les données et détecter les colonnes
all_rows = []
all_fieldnames = set()

# Boucle principale pour traiter chaque jour
current_date = start_date
while current_date < end_limit:
    # Define the date window (1 day per request)
    end_date = current_date + timedelta(days=1)
    
    # Format dates as ISO 8601 (YYYY-MM-DD)
    start_str = current_date.strftime("%Y-%m-%d")
    end_str = end_date.strftime("%Y-%m-%d")
    
    # Construct the API URL
    url = (
        f"{base_url}?dataset={dataset}"
        f"&startDate={start_str}"
        f"&endDate={end_str}"
    )
    
    print(f"API call: {start_str} → {end_str}")
    
    try:
        response = requests.get(url)
        
        # Check HTTP response
        if response.status_code == 200:
            json_full = response.json()
            data = json_full.get("data", [])
            
            # Loop over each municipality record
            for item in data:
                if not isinstance(item, dict):
                    continue
                
                # Structure de base avec les champs de niveau supérieur
                base_row = {
                    "type": item.get("type"),
                    "id": item.get("id")
                }
                
                attr = item.get("attributes", {})
                
                # Ajouter automatiquement tous les attributs (sauf les données imbriquées)
                for attr_key, attr_value in attr.items():
                    if attr_key != "installedCapacityPerMeteringPointTypeGroupMunicipalityDaily":
                        base_row[attr_key] = attr_value
                
                # Loop over each installed capacity daily record
                for record in attr.get("installedCapacityPerMeteringPointTypeGroupMunicipalityDaily", []):
                    # Créer une ligne complète en combinant les infos de base et les données quotidiennes
                    row = {**base_row}  # Copier les données de base
                    
                    # Ajouter automatiquement toutes les données du record
                    for record_key, record_value in record.items():
                        row[record_key] = record_value
                    
                    # Ajouter toutes les clés trouvées à notre ensemble d'en-têtes
                    all_fieldnames.update(row.keys())
                    all_rows.append(row)
            
            if data:
                print(f"  {len(data)} municipality records processed for {start_str}")
            else:
                print(f"  No data available for {start_str}")
                
        else:
            print(f"API call failed: HTTP {response.status_code}")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"API error: {e}")
    
    # Move to the next day
    current_date = end_date
    time.sleep(1)  # Avoid rate limiting

# Écrire toutes les données dans le fichier CSV avec les en-têtes détectées automatiquement
if all_rows:
    with open(file_name, mode="w", newline="", encoding="utf-8") as file:
        # Trier les en-têtes pour avoir un ordre cohérent
        sorted_fieldnames = sorted(all_fieldnames)
        writer = csv.DictWriter(file, fieldnames=sorted_fieldnames)
        writer.writeheader()
        writer.writerows(all_rows)
    
    print(f"\nData saved in CSV file: {file_name}")
    print(f"Automatically detected columns: {sorted_fieldnames}")
    print(f"Total rows written: {len(all_rows)}")
    
    # Afficher quelques statistiques utiles
    if all_rows:
        print(f"\nSample of detected fields:")
        sample_row = all_rows[0]
        for key, value in sample_row.items():
            print(f"  {key}: {value} ({type(value).__name__})")
else:
    print("No data found in the specified date range")

API call: 2025-07-15 → 2025-07-16
  343 municipality records processed for 2025-07-15
API call: 2025-07-16 → 2025-07-17
  343 municipality records processed for 2025-07-16
API call: 2025-07-17 → 2025-07-18
  343 municipality records processed for 2025-07-17
API call: 2025-07-18 → 2025-07-19
  343 municipality records processed for 2025-07-18

Data saved in CSV file: municipalities_installed_capacity.csv
Automatically detected columns: ['id', 'installedCapacity', 'lastUpdatedTime', 'meteringPointTypeCode', 'municipalityId', 'municipalityNumber', 'name', 'nameNo', 'productionGroup', 'type', 'usageDateId']
Total rows written: 3596

Sample of detected fields:
  type: municipality (str)
  id: 3428 (str)
  municipalityNumber: 3428 (str)
  name: Alvdal (str)
  nameNo: Alvdal (str)
  usageDateId: 20250715 (int)
  municipalityId: 3428 (str)
  meteringPointTypeCode: E19 (str)
  productionGroup: solar (str)
  installedCapacity: 1309.0 (float)
  lastUpdatedTime: 2025-07-18T08:28:43+02:00 (str)


In [ ]:

# Constants
dataset = "INSTALLED_CAPACITY_PER_METERING_POINT_TYPE_GROUP_MUNICIPALITY_DAILY"  # Dataset identifier in the API
base_url = "https://api.elhub.no/energy-data/v0/municipalities"   # Base URL for the API endpoint

# Start and end dates
start_date = datetime(2025, 7, 15) # Start date for data collection authorized by the API
end_limit = datetime.now()

# Output CSV file
file_name = "municipalities_installed_capacity.csv"  

# Collections pour stocker toutes les données et détecter les colonnes
all_rows = []
all_fieldnames = set()

# Boucle principale pour traiter chaque jour
current_date = start_date
while current_date < end_limit:
    # Define the date window (1 day per request)
    end_date = current_date + timedelta(days=1)
    
    # Format dates as ISO 8601 (YYYY-MM-DD)
    start_str = current_date.strftime("%Y-%m-%d")
    end_str = end_date.strftime("%Y-%m-%d")
    
    # Construct the API URL
    url = (
        f"{base_url}?dataset={dataset}"
        f"&startDate={start_str}"
        f"&endDate={end_str}"
    )
    
    print(f"API call: {start_str} → {end_str}")
    
    try:
        response = requests.get(url)
        
        # Check HTTP response
        if response.status_code == 200:
            json_full = response.json()
            data = json_full.get("data", [])
            
            # Loop over each municipality record
            for item in data:
                if not isinstance(item, dict):
                    continue
                
                # Structure de base avec les champs de niveau supérieur
                base_row = {
                    "type": item.get("type"),
                    "id": item.get("id")
                }
                
                attr = item.get("attributes", {})
                
                # Ajouter automatiquement tous les attributs (sauf les données imbriquées)
                for attr_key, attr_value in attr.items():
                    if attr_key != "installedCapacityPerMeteringPointTypeGroupMunicipalityDaily":
                        base_row[attr_key] = attr_value
                
                # Loop over each installed capacity daily record
                for record in attr.get("installedCapacityPerMeteringPointTypeGroupMunicipalityDaily", []):
                    # Créer une ligne complète en combinant les infos de base et les données quotidiennes
                    row = {**base_row}  # Copier les données de base
                    
                    # Ajouter automatiquement toutes les données du record
                    for record_key, record_value in record.items():
                        row[record_key] = record_value
                    
                    # Ajouter toutes les clés trouvées à notre ensemble d'en-têtes
                    all_fieldnames.update(row.keys())
                    all_rows.append(row)
            
            if data:
                print(f"  {len(data)} municipality records processed for {start_str}")
            else:
                print(f"  No data available for {start_str}")
                
        else:
            print(f"API call failed: HTTP {response.status_code}")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"API error: {e}")
    
    # Move to the next day
    current_date = end_date
    time.sleep(1)  # Avoid rate limiting

# Écrire toutes les données dans le fichier CSV avec les en-têtes détectées automatiquement
if all_rows:
    with open(file_name, mode="w", newline="", encoding="utf-8") as file:
        # Trier les en-têtes pour avoir un ordre cohérent
        sorted_fieldnames = sorted(all_fieldnames)
        writer = csv.DictWriter(file, fieldnames=sorted_fieldnames)
        writer.writeheader()
        writer.writerows(all_rows)
    
    print(f"\nData saved in CSV file: {file_name}")
    print(f"Automatically detected columns: {sorted_fieldnames}")
    print(f"Total rows written: {len(all_rows)}")
    
    # Afficher quelques statistiques utiles
    if all_rows:
        print(f"\nSample of detected fields:")
        sample_row = all_rows[0]
        for key, value in sample_row.items():
            print(f"  {key}: {value} ({type(value).__name__})")
else:
    print("No data found in the specified date range")

API call: 2021-01-01 → 2021-01-02
  701 records written for 2021-01-01
API call: 2021-01-02 → 2021-01-03
  701 records written for 2021-01-02
API call: 2021-01-03 → 2021-01-04
  701 records written for 2021-01-03
API call: 2021-01-04 → 2021-01-05
  701 records written for 2021-01-04
API call: 2021-01-05 → 2021-01-06
  701 records written for 2021-01-05
API call: 2021-01-06 → 2021-01-07
  701 records written for 2021-01-06
API call: 2021-01-07 → 2021-01-08
  701 records written for 2021-01-07
API call: 2021-01-08 → 2021-01-09
  701 records written for 2021-01-08
API call: 2021-01-09 → 2021-01-10
  701 records written for 2021-01-09
API call: 2021-01-10 → 2021-01-11
  701 records written for 2021-01-10
API call: 2021-01-11 → 2021-01-12
  702 records written for 2021-01-11
API call: 2021-01-12 → 2021-01-13
  702 records written for 2021-01-12
API call: 2021-01-13 → 2021-01-14
  702 records written for 2021-01-13
API call: 2021-01-14 → 2021-01-15
  702 records written for 2021-01-14
API ca

In [9]:
df_capacity=pd.read_csv('municipalities_installed_capacity.csv')


In [11]:
df_capacity

,type,id,municipalityNumber,name,nameNo,usageDateId,municipalityId,meteringPointTypeCode,productionGroup,installedCapacity,lastUpdatedTime
0,municipality,5632,5632,Båtsfjord,Båtsfjord,20210101,5632,E18,wind,120000.0,2025-06-02T08:28:23+02:00
1,municipality,1563,1563,Sunndal,Sunndal,20210101,1563,E18,hydro,514585.0,2025-06-02T08:28:23+02:00
2,municipality,1563,1563,Sunndal,Sunndal,20210101,1563,E19,solar,15.0,2025-06-02T08:28:23+02:00
3,municipality,1836,1836,Rødøy,Rødøy,20210101,1836,E18,hydro,93671.0,2025-06-02T08:28:23+02:00
4,municipality,1836,1836,Rødøy,Rødøy,20210101,1836,E19,solar,12.0,2025-06-02T08:28:23+02:00
...,...,...,...,...,...,...,...,...,...,...,...
1271167,municipality,4212,4212,Vegårshei,Vegårshei,20250602,4212,E19,solar,267.0,2025-06-02T14:27:27+02:00
1271168,municipality,4633,4633,Fedje,Fedje,20250602,4633,E19,solar,38.0,2025-06-02T14:27:27+02:00
1271169,municipality,3234,3234,Lunner,Lunner,20250602,3234,E19,solar,1730.0,2025-06-02T14:27:27+02:00
1271170,municipality,3454,3454,Vang,Vang,20250602,3454,E19,solar,549.0,2025-06-02T14:27:27+02:00
